You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current landscape of Artificial Intelligence
- Importance of staying up-to-date with the latest trends and news in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing developments
- Deep learning breakthroughs
- Ethical considerations in AI

2. Key Players in Artificial Intelligence:
- Google Brain
- OpenAI
- IBM Watson
- Microso

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become an integral part of our daily lives, from personalized recommendations on streaming services to autonomous vehicles. As the technology continues to evolve at a rapid pace, it is crucial for professionals in the tech industry, AI enthusiasts, and students studying AI to stay up-to-date with the latest trends and news in the field. By understanding the current landscape of AI, individuals can make informed decisions and contribute to the advancements in this innovative technology.

## Latest Trends in Artificial Intelligence
In recent years, AI has seen significant advancements in machine learning, natural language processing, and deep learning. Machine learning algorithms have become increasingly sophisticated, allowing for more accurate predictions and insights. Natural language processing has also made great strides, e

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become an integral part of our daily lives, from personalized recommendations on streaming services to autonomous vehicles. As the technology continues to evolve at a rapid pace, it is crucial for professionals in the tech industry, AI enthusiasts, and students studying AI to stay up-to-date with the latest trends and news in the field. By understanding the current landscape of AI, individuals can make informed decisions and contribute to the advancements in this innovative technology.

## Latest Trends in Artificial Intelligence
In recent years, AI has seen significant advancements in machine learning, natural language processing, and deep learning. Machine learning algorithms have become increasingly sophisticated, allowing for more accurate predictions and insights. Natural language processing has also made great strides, enabling AI systems to understand and generate human language more effectively. Deep learning breakthroughs have revolutionized how AI models are trained, leading to improved performance across various tasks. Alongside these technical advancements, ethical considerations in AI have gained prominence, with discussions on bias, transparency, and accountability becoming essential in the development and deployment of AI systems.

## Key Players in Artificial Intelligence
Several key players in the AI industry have been at the forefront of innovation and research. Google Brain, known for its work in deep learning and neural networks, has been a driving force behind many AI breakthroughs. OpenAI, a research organization focused on developing safe and beneficial AI, has made significant contributions to the field. IBM Watson, with its cognitive computing capabilities, has been instrumental in advancing AI applications in various industries. Microsoft Cognitive Services offers a suite of AI tools and services that empower developers to build intelligent applications.

## Noteworthy News in Artificial Intelligence
Staying informed about the latest news in AI is essential for understanding the impact of the technology on society and industry. Recent research findings have shed light on new AI algorithms and applications, pushing the boundaries of what is possible with AI. Industry applications of AI, from healthcare to finance, continue to demonstrate the transformative power of the technology. Policy developments related to AI regulation are also shaping the future of AI, with governments and organizations working to establish guidelines for the responsible use of AI.

## Conclusion and Call to Action
In conclusion, Artificial Intelligence is a dynamic and rapidly evolving field that holds immense potential for innovation and progress. By keeping up with the latest trends, key players, and news in AI, individuals can stay informed and contribute to the responsible development of AI technologies. As the industry continues to grow and evolve, it is crucial for professionals, enthusiasts, and students to engage with the latest advancements in AI and drive the future of this transformative technology. Remember to stay informed, stay curious, and continue to explore the endless possibilities of Artificial Intelligence. Stay updated on the latest Artificial Intelligence trends, industry news, and key players in AI. Explore the latest advancements in machine learning and stay informed about the future implications of AI. By following this comprehensive content plan, the blog article on Artificial Intelligence will provide valuable insights and information to the target audience, helping them navigate the complex and ever-changing landscape of AI with confidence and understanding.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Multi-Agent Systems in Retail Industry"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Multi-Agent Systems in Retail Industry.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Multi-Agent Systems in Retail Industry

Outline:
I. Introduction
   A. Definition of Multi-Agent Systems
   B. Importance of Multi-Agent Systems in Retail Industry

II. Latest Trends in Multi-Agent Systems in Retail Industry
   A. Integration of AI and Machine Learning
   B. Personalized Customer Experiences
   C. Inventory Management Optimization

III. Key Players in Multi-Agent Systems in Retail Industry
   A. IBM
   B. Amazon
   C. Walmart

IV. Noteworthy News in Multi-Agent Sy

I now can give a great answer

Final Answer:
# Multi-Agent Systems in Retail Industry: Revolutionizing the Shopping Experience

## Introduction
In the rapidly evolving world of retail, innovation is key to success. Multi-Agent Systems (MAS) have emerged as a cutting-edge technology that is reshaping the retail industry. By leveraging multiple intelligent agents working together towards a common goal, MAS is revolutionizing customer experiences and operational efficiency in retail businesses.

## Latest Trends in Multi-Agent Systems in Retail Industry
### Integration of AI and Machine Learning
The integration of artificial intelligence (AI) and machine learning into MAS has enabled retailers to gain valuable insights from real-time data analysis. This allows for personalized customer experiences, targeted marketing strategies, and seamless shopping journeys that cater to individual preferences and behaviors.

### Personalized Customer Experiences
MAS empowers retailers to deliver person

In [14]:
Markdown(result)

# Multi-Agent Systems in Retail Industry: Revolutionizing the Shopping Experience

## Introduction
In the rapidly evolving world of retail, innovation is key to success. Multi-Agent Systems (MAS) have emerged as a cutting-edge technology that is reshaping the retail industry. By leveraging multiple intelligent agents working together towards a common goal, MAS is revolutionizing customer experiences and operational efficiency in retail businesses.

## Latest Trends in Multi-Agent Systems in Retail Industry
### Integration of AI and Machine Learning
The integration of artificial intelligence (AI) and machine learning into MAS has enabled retailers to gain valuable insights from real-time data analysis. This allows for personalized customer experiences, targeted marketing strategies, and seamless shopping journeys that cater to individual preferences and behaviors.

### Personalized Customer Experiences
MAS empowers retailers to deliver personalized customer experiences through AI-powered algorithms that analyze customer data and tailor product recommendations, promotions, and pricing strategies. This level of personalization not only enhances customer satisfaction but also fosters loyalty and retention rates, leading to long-term business growth.

### Inventory Management Optimization
Efficient inventory management is crucial for retailers to meet customer demand while optimizing costs. MAS facilitates inventory optimization by analyzing sales data, forecasting demand, and automating replenishment processes. This results in reduced stockouts, improved supply chain efficiency, and increased profitability for retail businesses.

## Key Players in Multi-Agent Systems in Retail Industry
### IBM
IBM leads the MAS space by offering advanced solutions that improve operational efficiency, drive sales growth, and enhance customer satisfaction. With a strong focus on AI and machine learning technologies, IBM empowers retailers to make data-driven decisions and stay ahead of the competition in a rapidly changing retail landscape.

### Amazon
As a pioneer in e-commerce and retail innovation, Amazon has leveraged MAS to transform the shopping experience for millions of customers globally. From personalized product recommendations to dynamic pricing strategies, Amazon's MAS capabilities have set new standards for customer engagement and loyalty in the retail industry.

### Walmart
Walmart, a retail giant known for its commitment to technological innovation, has embraced MAS to optimize its supply chain, improve inventory management, and enhance the overall shopping experience. By utilizing AI-powered agents, Walmart can anticipate customer demand, streamline logistics, and drive operational excellence across its extensive network of stores.

## Noteworthy News in Multi-Agent Systems in Retail Industry
### Case Studies
Leading retailers have successfully implemented MAS to achieve significant improvements in sales, customer satisfaction, and operational efficiency. Case studies from industry pioneers such as IBM and Amazon demonstrate the tangible benefits of MAS in driving business growth and gaining a competitive edge in the retail sector.

### Industry Reports
Reputable sources like Gartner and Forrester provide valuable insights into the latest trends, developments, and best practices in MAS for retail. These industry reports highlight the transformative impact of MAS on the retail sector and offer strategic guidance for retailers seeking to harness the power of intelligent agents for success.

### Technology Updates
Stay informed on the latest technology advancements in MAS for retail by following industry blogs and publications. From AI-driven chatbots to predictive analytics tools, retailers can explore innovative solutions that leverage MAS to enhance customer engagement, streamline operations, and increase profitability.

## Conclusion
As retail continues to undergo digital transformation, the adoption of Multi-Agent Systems has become essential for businesses seeking to thrive in a competitive market. By harnessing the capabilities of AI, machine learning, and intelligent agents, retailers can unlock new opportunities to elevate customer experiences, optimize operations, and achieve sustainable success. To learn more about how MAS can revolutionize your retail business, contact us for a consultation today.

This blog post on Multi-Agent Systems in the Retail Industry offers valuable insights and practical recommendations for retail professionals, tech enthusiasts, and business owners aiming to drive innovation in the digital era. By staying informed on the latest trends, key industry players, and noteworthy news in MAS, retailers can position themselves for success and establish a sustainable competitive advantage in a dynamic market environment.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).